<a href="https://colab.research.google.com/github/mahyarhabibi/GenderGaps_Hollywood/blob/main/Codes/ETL_Patent/Data_2022/KyWrd_search/KyWrd_corp_quarter_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


## Construct Corp-Quarter Level ML Patent Data.

In [ ]:
data_dir =  '/content/gdrive/MyDrive/Research/AI_innovation/Data/Patents/ML_Patents_KyWrd'

In [ ]:
# Loading and Pre-Processing
df_mlpat = pd.read_parquet(os.path.join(data_dir, 
                                        'KyWrd_patents_merged_info.parquet'))

#spelling mistake in column
df_mlpat = df_mlpat.rename(columns={'assing_addr': 'assign_addr'})

df_mlpat['patent_date'] = pd.to_datetime(df_mlpat['patent_date'],
                                        format = "%Y-%m-%d")

df_mlpat['filing_date'] = pd.to_datetime(df_mlpat['filing_date'],
                                        format = "%Y-%m-%d")
df_mlpat['patent_year'] = df_mlpat['patent_date'].dt.year
df_mlpat['file_year'] = df_mlpat['filing_date'].dt.year
df_mlpat['file_quart'] = df_mlpat['filing_date'].dt.quarter
# Remove Obs filed before 2010 or after 2019
df_mlpat = df_mlpat.loc[(df_mlpat['file_year']>=2010) &
                     (df_mlpat['file_year']<=2019)]

# Remove withdrawn patents
df_mplat = df_mlpat[df_mlpat['withdrawn']==0]     
df_mlpat.drop(columns='withdrawn', inplace=True)

# Remove Reissue patents
df_mlpat = df_mlpat[ df_mlpat['patent_type']!='reissue']
df_mlpat.drop(columns='patent_type', inplace=True)

# Reset Index
df_mlpat  = df_mlpat.reset_index(drop=True)

# count citations
df_mlpat['citation'] = df_mlpat['cited_by'].apply(lambda x: len(x) if x is not None
                                                  else 0)


In [ ]:
# The subset assigned to organizations
df_patorg = df_mlpat.loc[~ df_mlpat['assign_org'].isna()].reset_index(drop=True)


# explode the patents assigned to multiple orgs

keepvars = ['patent_id', 'patent_year', 'file_year', 'file_quart',
            'kyw_desc_dl', 'kyw_desc_ml', 'kyw_desc_app', 'kyw_sum_dl',
            'kyw_sum_ml', 'kyw_sum_app', 'assign_id', 'assign_org',
            'assign_type', 'assign_addr','citation' ]

df_corps = df_patorg[keepvars].explode(['assign_id', 'assign_org',
                                        'assign_type', 'assign_addr'])

# Keep US or Int. corporations
df_corps = df_corps[((df_corps['assign_type']==2 )|
                     (df_corps['assign_type']==3)) ].reset_index(drop=True)

# Location
df_corps['country'] = df_corps['assign_addr'].apply(lambda x: x['country'])
df_corps['state'] = df_corps['assign_addr'].apply(lambda x: x['state'])
df_corps['city'] = df_corps['assign_addr'].apply(lambda x: x['city'])

In [ ]:
# Data of unique corps names and locations per assign id 
corpvar = ['assign_id', 'file_year' ,'assign_org', 'country', 'state',
                          'city']

df_corps_char = df_corps[corpvar].sort_values(by=['assign_id', 'file_year'],
                                              ascending=True)

df_corps_char = df_corps_char.drop_duplicates(subset='assign_id', keep='last')
df_corps_char = df_corps_char.drop(columns='file_year')

In [ ]:
# Aggregate Patent data
df_corps_gp = df_corps.groupby(by=['assign_id','file_year',  'file_quart'])\
                                   ['kyw_desc_dl','kyw_desc_ml', 'kyw_desc_app',
                                    'kyw_sum_dl', 'kyw_sum_ml','kyw_sum_app',
                                    'citation'].sum().reset_index()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
# simple use of groupby ommites quarter when a corp has not applied for any patents
# create complete corp-quarter data
df_yr_qrt = df_corps[['file_year', 'file_quart']].drop_duplicates().\
            reset_index(drop=True)
assert len(df_yr_qrt)==40

df_corp_qrt = df_corps_char.merge(df_yr_qrt, how='cross')

# merged with grouped corporate-quarter data
df_corp_qrt = df_corp_qrt.merge(df_corps_gp, on=['assign_id',  'file_year', 
                                                 'file_quart'], how='left')

In [ ]:
# fill missing values with zero
cols = ['kyw_desc_dl', 'kyw_desc_ml','kyw_desc_app', 'kyw_sum_dl',
             'kyw_sum_ml', 'kyw_sum_app', 'citation']
for col in cols:
  df_corp_qrt[col] = df_corp_qrt[col].fillna(0)

# rename columns
df_corp_qrt.rename(columns={'kyw_desc_dl': 'desc_dl', 'kyw_desc_ml': 'desc_ml',
                            'kyw_desc_app': 'desc_app','kyw_sum_dl': 'sum_dl',
                            'kyw_sum_ml': 'sum_ml','kyw_sum_app': 'sum_app'},
                   inplace=True)

# save dataframe
df_corp_qrt.to_parquet(os.path.join(data_dir, 'KyWrd_corp_quarter.parquet'))

## Corp-Quarter All Corps All Patents

In [ ]:
app_data = '/content/gdrive/MyDrive/Research/USPTO/Patents/Patents_Update2022/g_application.tsv.zip'
assignee_data = '/content/gdrive/MyDrive/Research/USPTO/Patents/Patents_Update2022/g_assignee_disambiguated.tsv.zip'
UScite_data = '/content/gdrive/MyDrive/Research/USPTO/Patents/Patents_Update2022/g_us_patent_citation.tsv.zip'
patent_data = '/content/gdrive/MyDrive/Research/USPTO/Patents/Patents_Update2022/g_patent.tsv.zip'

In [ ]:
# Applications
df_app = pd.read_csv(app_data, sep='\t', usecols=['patent_id', 'filing_date'],
                      dtype={'patent_id': object})

df_app['filing_date'] = pd.to_datetime(df_app['filing_date'],
                                        format = "%Y-%m-%d", errors='coerce')
df_app['file_year'] = df_app['filing_date'].dt.year
df_app = df_app[df_app['file_year']>=2010].reset_index(drop=True)

# Patents
df_pat = pd.read_csv(patent_data, sep='\t', usecols=['patent_id', 'patent_type',
                      'withdrawn'], dtype={'patent_id': object})

# Merge and clean
df_pat_app = df_app.merge(df_pat, on='patent_id')
df_pat_app =df_pat.copy()
df_pat_app = df_pat_app[df_pat_app['withdrawn']==0]
df_pat_app = df_pat_app[df_pat_app['patent_type']!='reissue']
df_pat_app.drop(columns='withdrawn', inplace=True)

In [ ]:
# Assignee data
df_ass = pd.read_csv(assignee_data, sep='\t', dtype={'patent_id':  object},
                     usecols=['patent_id','assignee_id','assignee_type',
                              'disambig_assignee_organization']).rename(
                                  columns={ 'disambig_assignee_organization':
                                    'org_name'} )

# keep corps
df_ass = df_ass[((df_ass['assignee_type']==2) | (df_ass['assignee_type']==3))]
df_ass.drop(columns='assignee_type', inplace=True)
df_pat_org = df_pat_app.merge(df_ass, on='patent_id', how='inner')

In [ ]:
# How many times each patent has been cited
data_cite = pd.read_csv(UScite_data, sep='\t', iterator=True, chunksize=1000000,
                        usecols=['patent_id', 'citation_patent_id'], 
                        dtype={'patent_id': object,
                               'citation_patent_id': object})

patent_ids = df_pat_org['patent_id'].values.tolist()
df_cite = pd.DataFrame(columns=['patent_id', 'citation_patent_id'])

for n, chunk in enumerate(data_cite):
  df_temp = chunk[chunk['citation_patent_id'].isin(patent_ids)]
  df_cite = df_cite.append(df_temp)

df_cite_gp = df_cite.groupby('citation_patent_id')['patent_id'].count().reset_index()
df_cite_gp.rename(columns={'citation_patent_id': 'patent_id', 
                           'patent_id': 'N_citation'}, inplace=True)

df_pat_org = df_pat_org.merge(df_cite_gp, on='patent_id', how='left')
df_pat_org['N_citation'].fillna(0, inplace=True)

In [ ]:
# Save DataFrame
output_dir = '/content/gdrive/MyDrive/Research/AI_innovation/Data/Patents'
df_pat_org.to_parquet(os.path.join(output_dir, 'Corps_all_patents.parquet'))

In [ ]:
df_pat_org = pd.read_parquet(os.path.join(output_dir, 'Corps_all_patents.parquet'))

In [ ]:
# aggregate to corp-quarter level
df_pat_org['file_quart'] = df_pat_org['filing_date'].dt.quarter

In [ ]:
# Collapse Patent data to Corp-Quarter level
df_pat_org_gp = df_pat_org.groupby(['assignee_id', 'file_year', 'file_quart']).\
                agg({'patent_id':'count', 'N_citation': 'sum'}).reset_index().\
                rename(columns={'patent_id': 'N_patent'}) 
                
# Data of unique corps names and locations per assign id 
corpvar = ['assignee_id', 'file_year', 'org_name']

df_corps_char = df_pat_org[corpvar].sort_values(by=['assignee_id', 'file_year'],
                                              ascending=True)

df_corps_char = df_corps_char.drop_duplicates(subset='assignee_id', keep='last')
df_corps_char = df_corps_char.drop(columns='file_year')
# create complete corp-quarter base dataset
df_yr_qrt = df_pat_org[['file_year', 'file_quart']].drop_duplicates().\
            reset_index(drop=True)

df_corp_qrt = df_corps_char.merge(df_yr_qrt, how='cross')

# merge with grouped corporate-quarter data
df_corp_qrt = df_corp_qrt.merge(df_pat_org_gp, on=['assignee_id',  'file_year', 
                                                 'file_quart'], how='left')
# fill missing values with zero
cols = ['N_patent', 'N_citation']
for col in cols:
  df_corp_qrt[col] = df_corp_qrt[col].fillna(0)

# save dataframe
df_corp_qrt.to_parquet(os.path.join(output_dir, 'Corp_quarter_all_patents.parquet'))